In [ ]:
from ase.db import connect
import pandas as pd
from ase.data import chemical_symbols
import ase
import jarvis.core as js

db = connect('c2db-2022-11-30.db')
mats=list(db.select())

data=[m.__dict__ for m in mats]
dt=pd.DataFrame(data)
dt=dt[[c for c in dt.columns if not c.startswith('_')]]

In [ ]:
import numpy as np
from numpy.linalg import norm

#example of grabbing a subset of c2db
dt_nona=dt[~dt["is_magnetic"].isna()]
dt_mag=dt_nona[dt_nona["is_magnetic"]]
dt_mag["A"]=dt_mag["cell"].apply(lambda x: norm(x[0]))
dt_mag[(dt_mag["A"]>=4.3)&(dt_mag["A"]<=4.4)][["uid","A","gap"]]

In [ ]:
def get_alignn_atoms(x):
    js_atoms=js.atoms.ase_to_atoms(ase.Atoms(x.numbers,x.positions,cell=x.cell,pbc=x.pbc))
    return dict(js_atoms.to_dict())

In [ ]:
%mkdir c2db_folder

In [ ]:
dt["poscar_atoms"]=dt.apply(lambda x: get_alignn_atoms(x),axis=1)
dt["POSCAR_filename"]="POSCAR-"+dt["uid"]+".vasp"
dt[["POSCAR_filename","cell_area"]].to_csv("c2db_area_predict/id_prop.csv",index=False,header=False)

for i,x in dt.iterrows():
    atoms=js.atoms.Atoms.from_dict(x.distorted_atoms)
    atoms.write_poscar("c2db_folder/"+x.POSCAR_filename)